In [31]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import folium

In [32]:
fema = gpd.read_file("Data/FEMA_6/FEMA_6.shp")
fema = fema.sort_values(by='CTFIPS')

In [36]:
def ag_count(state):
    data = gpd.read_file("Data/SoVI2010_" + state + "/SoVI0610_" + state + ".shp")
    data['CTFIPS'] = data['STATEFP10'] + data['COUNTYFP10']
    data = data.groupby('CTFIPS').agg({"SOVI0610" + state: "mean", "P0010001": "sum"})
    data = fema.merge(data, on="CTFIPS", how="right")
    data = data.rename(columns={
        "SOVI0610" + state: "SOVI",
        "P0010001": "POPULATION",
        "TOTAL_DECL": "TOTAL_DECLARATIONS"
    })
    data = data.sort_values(by='CTFIPS')
    data['SOVI_RANK'] = data['SOVI'].rank()
    rank = []
    counties = []
    n = len(data)   
    for index, row in data.iterrows():
        counties.append(row.CTFIPS[-3:])
        if row.SOVI_RANK < (n/3):
            rank.append(1)
        elif row.SOVI_RANK < (2 * (n/3)):
            rank.append(2)
        else:
            rank.append(3)
    data['COUNTYFP'] = counties
    data['RANK'] = rank
    return data

CA = ag_count("CA")
FL = ag_count("FL")
IN = ag_count("IN")
LA = ag_count("LA")
ME = ag_count("ME")
WA = ag_count("WA")
CA

,OBJECTID,STFIPS,ST_ABBR,CTFIPS,STATE,COUNTY,GEO_SUFFIX,TOTAL_DECLARATIONS,FLOOD_1964,ICE_STORM_,...,Shape_Leng,Winter_196,Shape_Le_1,Shape_Area,geometry,SOVI,POPULATION,SOVI_RANK,COUNTYFP,RANK
0,58,06,CA,06001,California,Alameda,County,14,3,0,...,3.610286,1,3.610286,0.196849,(POLYGON ((-121.9673689779958 37.4676361297038...,-0.174812,1510271.0,19.0,001,1
1,39,06,CA,06003,California,Alpine,County,6,3,0,...,2.174212,0,2.174212,0.199044,POLYGON ((-119.5854370003171 38.71321199963438...,0.686160,1175.0,27.0,003,2
2,21,06,CA,06005,California,Amador,County,9,3,0,...,2.909742,0,2.909742,0.161954,POLYGON ((-120.0723920002904 38.70276699999727...,1.558083,38091.0,42.0,005,3
3,53,06,CA,06007,California,Butte,County,10,3,0,...,4.205838,1,4.205838,0.455942,POLYGON ((-121.0766949997742 39.59726399983401...,1.954188,220000.0,48.0,007,3
4,22,06,CA,06009,California,Calaveras,County,8,1,0,...,2.963056,0,2.963056,0.276236,POLYGON ((-120.0199510002383 38.43352099969468...,1.929986,45578.0,47.0,009,3
5,16,06,CA,06011,California,Colusa,County,10,3,0,...,3.212898,1,3.212898,0.312200,POLYGON ((-121.8900129998999 39.38386400012371...,0.455662,21419.0,24.0,011,2
6,50,06,CA,06013,California,Contra Costa,County,11,4,0,...,3.019489,0,3.019489,0.201071,(POLYGON ((-122.3579787384279 37.9011267856727...,-0.402607,1049025.0,14.0,013,1
7,111,06,CA,06015,California,Del Norte,County,12,5,0,...,3.639225,0,3.639225,0.283927,(POLYGON ((-124.0310973038972 41.5178907133859...,3.393793,28610.0,58.0,015,3
8,56,06,CA,06017,California,El Dorado,County,8,3,0,...,3.792264,0,3.792264,0.479623,POLYGON ((-119.9043149997373 38.93332400040248...,-0.455807,181058.0,13.0,017,1
9,38,06,CA,06019,California,Fresno,County,9,3,0,...,9.004000,3,9.004000,1.571173,POLYGON ((-118.7750139996962 37.46305200014984...,1.159163,924798.0,39.0,019,3


In [34]:
sovi = CA.append([FL, IN, LA, ME, WA])
sovi = sovi.sort_values(by='CTFIPS')
sovi.head()

,OBJECTID,STFIPS,ST_ABBR,CTFIPS,STATE,COUNTY,GEO_SUFFIX,TOTAL_DECLARATIONS,FLOOD_1964,ICE_STORM_,...,Shape_Leng,Winter_196,Shape_Le_1,Shape_Area,geometry,SOVI,POPULATION,SOVI_RANK,COUNTYFP,RANK
0,58,06,CA,06001,California,Alameda,County,14,3,0,...,3.610286,1,3.610286,0.196849,(POLYGON ((-121.9673689779958 37.4676361297038...,-0.174812,1510271.0,19.0,001,1
1,39,06,CA,06003,California,Alpine,County,6,3,0,...,2.174212,0,2.174212,0.199044,POLYGON ((-119.5854370003171 38.71321199963438...,0.686160,1175.0,27.0,003,2
2,21,06,CA,06005,California,Amador,County,9,3,0,...,2.909742,0,2.909742,0.161954,POLYGON ((-120.0723920002904 38.70276699999727...,1.558083,38091.0,42.0,005,3
3,53,06,CA,06007,California,Butte,County,10,3,0,...,4.205838,1,4.205838,0.455942,POLYGON ((-121.0766949997742 39.59726399983401...,1.954188,220000.0,48.0,007,3
4,22,06,CA,06009,California,Calaveras,County,8,1,0,...,2.963056,0,2.963056,0.276236,POLYGON ((-120.0199510002383 38.43352099969468...,1.929986,45578.0,47.0,009,3


In [38]:
save_sovi = sovi[[
    'CTFIPS',
    'TOTAL_DECLARATIONS',
    'SOVI',
    'POPULATION',
    'RANK',
    'COUNTYFP',
    'geometry'
]]
save_sovi.to_file("SOVI.shp", index=False)